# Configuration tutorial

This notebook shows the flexibility of the Config class and instantiate function in venturi, which allows instantiating any Python object you want inside a Python script from a yaml file or dictionary.

Usually, the Config class loads a configuration from a yaml file. But here we use a dictionary for better understanding of the class.

## Using as a simple namespace

A config object created from a dictionary or yaml file behaves like a Namespace object from argparse

In [1]:
from torch import nn

from venturi import Config, instantiate


class SimpleCNN(nn.Module):
    """Simple CNN model."""

    def __init__(self, in_channels, out_channels, base_channels=16, kernel_size=3):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels, base_channels, kernel_size),
            nn.Conv2d(base_channels, base_channels * 2, kernel_size),
            nn.Conv2d(base_channels * 2, out_channels, kernel_size),
        )

    def forward(self, x):
        return self.layers(x)


args_dict = {
    "in_channels": 3,
    "out_channels": 10,
    "base_channels": 16,
    "kernel_size": 3,
}
args = Config(args_dict)


model = SimpleCNN(
    in_channels=args.in_channels,
    out_channels=args.out_channels,
    base_channels=args.base_channels,
    kernel_size=args.kernel_size,
)

You can access elements with dot notation or use the object as a dictionary

In [2]:
print(args["kernel_size"])

3


## Instantiating objects

The special key *_target_* can be used to instantiate any class or function using the provided parameters at the same level of the key. 

In [3]:
args_dict = {
    "_target_": "SimpleCNN",
    "in_channels": 3,
    "out_channels": 10,
    "base_channels": 16,
    "kernel_size": 3,
}

cfg = Config(args_dict)
model = instantiate(cfg)
model

SimpleCNN(
  (layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (2): Conv2d(32, 10, kernel_size=(3, 3), stride=(1, 1))
  )
)

You can even create complex nested objects

In [4]:
class Classifier(nn.Module):
    """A simple classifier model that receives a CNN backbone in the constructor."""

    def __init__(self, cnn_model: nn.Module, num_classes: int):
        super().__init__()
        self.conv = cnn_model
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(cnn_model.layers[-1].out_channels, num_classes)

    def forward(self, x):
        x = self.conv(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


args_dict = {
    "_target_": "Classifier",  # Top class
    "cnn_model": {
        "_target_": "SimpleCNN",  # First argument of the Classifier class
        "in_channels": 3,
        "out_channels": 10,
    },
    "num_classes": 10,
}

cfg = Config(args_dict)
model = instantiate(cfg)
model

Classifier(
  (conv): SimpleCNN(
    (layers): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
      (2): Conv2d(32, 10, kernel_size=(3, 3), stride=(1, 1))
    )
  )
  (pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Linear(in_features=10, out_features=10, bias=True)
)

Note that this makes the dictionary define ALL information required to create the object. If you want to instantiate another class, **no code changes are required**. Just change the dictionary

In [5]:
args_dict = {
    "_target_": "torch.nn.Conv2d",
    "in_channels": 3,
    "out_channels": 10,
    "kernel_size": 3,
}

cfg = Config(args_dict)
model = instantiate(cfg)
model

Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))

This allows separating the specific classes, functions and parameters used in an experiment from the training loop. Is also allows the registration of every relevant information used in an experiment.

## Example: Data augmentation

For example, you can construct an entire image augmentation pypeline from a dictionary.

In [6]:
from torchvision.transforms import v2


class SegmentationTransform:
    """Image augmentation pipeline."""

    def __init__(self, cfg_transforms: Config):
        # Instantiate each transform and add to list
        transforms_list = []
        for t in cfg_transforms.values():
            transforms_list.append(instantiate(t))
        transforms = v2.Compose(transforms_list)

        self.transforms = transforms

    def __call__(self, image):
        return self.transforms(image)


# Define the transforms and their parameters
transforms = {
    "_target_": "SegmentationTransform",
    "cfg_transforms": {
        "rrc": {
            "_target_": "torchvision.transforms.v2.RandomResizedCrop",
            "size": [64, 64],
            "scale": [0.6, 1.0],
            "ratio": [0.75, 1.33],
            "antialias": True,
        },
        "rhf": {"_target_": "torchvision.transforms.v2.RandomHorizontalFlip", "p": 0.5},
        "rvf": {"_target_": "torchvision.transforms.v2.RandomVerticalFlip", "p": 0.5},
    },
}

trans_dict = Config(transforms)
transform_pipeline = instantiate(trans_dict)

It is also possible to do *lazy instantiation*, that is, create an object with *some* given parameters but only instatiate it later. 

In [7]:
args_dict = {
    "_target_": "torch.optim.SGD",
    "lr": 0.01,
    "momentum": 0.9,
}
cfg = Config(args_dict)
# Create a partial optimizer factory
optimizer_factory = instantiate(cfg, partial=True)
# Do whatever you want with it
...
# Actually create the optimizer with model parameters
optimizer = optimizer_factory(model.parameters())

Lazy instantiation also allows creating new functions by fixing some parameters of a given function

In [8]:
def power(x, exponent):
    return x**exponent


arg_dict = {"_target_": "power", "exponent": 2}

cfg = Config(arg_dict)
square_func = instantiate(cfg, partial=True)
# The line above is equivalent to:
# def square_func(x):
#     return power(x, exponent=2)
square_func(3)

9

## Mixing configurations

Configurations can be easily mixed with other yaml files or dictionaries

In [9]:
cfg = Config("config/example_config.yaml")
print("Base configuration")
print(cfg)

# Change the model to a Vision Transformer
cfg.model = Config("config/vit_config.yaml").model
print("New configuration")
print(cfg)

Base configuration
dataset:
  setup:
    _target_: MyDataset
  params:
    num_train_samples: 100
    num_val_samples: 20
    num_classes: 10
    img_size:
    - 224
    - 224
model:
  setup:
    _target_: MyCNNModel
  params:
    num_input_channels: 3
    num_classes: 10
    base_filters: 128
    num_layers: 50
    kernel_size: 3
losses:
  cross_entropy:
    instance:
      _target_: scripts.metrics.WeightedBCEWithLogitsLoss

New configuration
dataset:
  setup:
    _target_: MyDataset
  params:
    num_train_samples: 100
    num_val_samples: 20
    num_classes: 10
    img_size:
    - 224
    - 224
model:
  setup:
    _target_: MyViTModel
  params:
    num_input_channels: 3
    num_classes: 10
    hidden_size: 768
    patch_size: 16
    num_attention_heads: 12
losses:
  cross_entropy:
    instance:
      _target_: scripts.metrics.WeightedBCEWithLogitsLoss



In [10]:
# Change some parameters of the ViT model. The full hierarchy of the configuration must be
# respected.
new_args = {"model": {"params": {"hidden_size": 2048, "num_attention_heads": 36}}}
cfg.update_from_dict(new_args)
cfg

Config({'dataset': Config({'setup': Config({'_target_': 'MyDataset'}), 'params': Config({'num_train_samples': 100, 'num_val_samples': 20, 'num_classes': 10, 'img_size': [224, 224]})}), 'model': Config({'setup': Config({'_target_': 'MyViTModel'}), 'params': Config({'num_input_channels': 3, 'num_classes': 10, 'hidden_size': 2048, 'patch_size': 16, 'num_attention_heads': 36})}), 'losses': Config({'cross_entropy': Config({'instance': Config({'_target_': 'scripts.metrics.WeightedBCEWithLogitsLoss'})})})})

The final configuration can be saved to a yaml file

In [11]:
cfg.save("config/final_config.yaml")

It is possible to define a configuration object with multiple models, and instantiate each inside a hyperparameter search

In [12]:
args_dict = {
    "models": {
        "resnet18": {
            "_target_": "torchvision.models.resnet18"
            # Model parameters can be added here
        },
        "vit": {
            "_target_": "torchvision.models.vit_b_16"
            # Model parameters can be added here
        },
    }
}
cfg = Config(args_dict)
for model_cfg in cfg.models.values():
    model = instantiate(model_cfg)
    ...

Note that this allows adding or removing models with no code changes